Before running this notebook, you need to compute head influence (the distance of noisy word representations before and after masking a specific head). You can run the following to do so:

`Python ./representations/masked_head_distance.py`

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.metrics import accuracy_score

def get_head_accuracy(ref_heads, pred_heads):
    layer_acc = []
    for i in range(ref_heads.shape[-1]):
        layer_acc.append(accuracy_score(ref_heads[:, i], pred_heads[:, i]))
        
    return layer_acc

### Robustness and Influential Heads Accuracy

In [ ]:
plt.rcParams['figure.dpi'] = 300
sns.set_theme(style='whitegrid', font_scale=1.4)

directions = ['en-es', 'fr-es', 'en-de', 'en-it', 'en-nl']
errors = ['morpheus']
fig_path = f'../results/figures/Robustness_Influential_Accuracy'



combined_df_list = []
for direction in directions:
    src = direction.split('-')[0]
    tgt = direction.split('-')[1]

    models = {
        f'opus-{direction}': f'opus-mt-{src}-{tgt}',
        f'm2m100-{direction}': 'm2m100_418M', 
        f'mbart-{direction}': 'mbart-large-50-many-to-many-mmt',
        f'nllb-{direction}': 'nllb-200-distilled-600M'
    }
    
    for finetuned_model, base_model in models.items():
        for error in errors:
            with open(f'../outputs/representations/head_masking/{direction}/{finetuned_model}-{error}/last/test.{error}.{src}.distance.pkl', 'rb') as f:
                noisy_output = pickle.load(f)
                
            with open(f'../outputs/representations/head_masking/{direction}/{finetuned_model}-{error}/last/test.{error}.{src}.to_clean.distance.pkl', 'rb') as f:
                noisy_output_to_clean = pickle.load(f)

            with open(f'../outputs/representations/head_masking/{direction}/{finetuned_model}-clean-{error}/last/test.{error}.{src}.distance.pkl', 'rb') as f:
                clean_output = pickle.load(f)
                
            with open(f'../outputs/representations/head_masking/{direction}/{finetuned_model}-clean-{error}/last/test.{error}.{src}.to_clean.distance.pkl', 'rb') as f:
                clean_output_to_clean = pickle.load(f)

            with open(f'../outputs/representations/head_masking/{direction}/{base_model}/last/test.{error}.{src}.distance.pkl', 'rb') as f:
                base_output = pickle.load(f)
                
            with open(f'../outputs/representations/head_masking/{direction}/{base_model}/last/test.{error}.{src}.to_clean.distance.pkl', 'rb') as f:
                base_output_to_clean = pickle.load(f)

            noisy_influence_heads = noisy_output.argmax(-1)
            noisy_influence_heads_to_clean = noisy_output_to_clean.argmax(-1)
            
            clean_influence_heads = clean_output.argmax(-1)
            clean_influence_heads_to_clean = clean_output_to_clean.argmax(-1)
            
            base_influence_heads = base_output.argmax(-1)
            base_influence_heads_to_clean = base_output_to_clean.argmax(-1)
            
            noisy_heads_acc = get_head_accuracy(noisy_influence_heads_to_clean, noisy_influence_heads)
            clean_heads_acc = get_head_accuracy(clean_influence_heads_to_clean, clean_influence_heads)
            base_heads_acc = get_head_accuracy(base_influence_heads_to_clean, base_influence_heads)
            
            n_heads = noisy_output.shape[-1]
            n_layers = noisy_output.shape[-2]

            model_names = ["Noise Finetuned"] * n_layers + ["Clean Finetuned"] * n_layers + ["Base"] * n_layers
            df = pd.DataFrame({
                "Head Accuracy": noisy_heads_acc + clean_heads_acc + base_heads_acc,
                "Model": model_names,
                "Layer": list(np.arange(1, n_layers+1)) * 3,
                "Error": [error] * (n_layers * 3),
                "Model Prefix": [finetuned_model.split('-')[0]] * (n_layers * 3)
            })

            df['Language'] = '-'.join([d.capitalize() for d in direction.split('-')])
            combined_df_list.append(df)

combined_df = pd.concat(combined_df_list)
combined_df.loc[combined_df['Model Prefix']=='opus','Model Prefix'] = 'OPUS-MT'
combined_df['Model Prefix'] = combined_df['Model Prefix'].apply(lambda x: x.upper())
combined_df['Error'] = combined_df['Error'].apply(lambda x: x.capitalize())

g = sns.catplot(
    x='Layer', 
    y='Head Accuracy', 
    hue='Model', 
    row='Model Prefix',  
    col='Language',       
    data=combined_df, 
    kind='point', 
    height=5, 
    aspect=1.2,
    palette='Set2',
    sharey=True
)

for ax in g.axes.flat:
    ax.tick_params(axis='x', labelsize=20)
    ax.tick_params(axis='y', labelsize=20)

sns.move_legend(
    g, "lower center",
    bbox_to_anchor=(0.5, -0.03), ncol=3, title=None, frameon=False,
    )

plt.ylim(0.0, 1.0)
plt.subplots_adjust(hspace=0.2, wspace=0.1)

plt.setp(g._legend.get_texts(), fontsize=30)
g.fig.suptitle(f"Accuracy of Robustness vs. Influential Heads: Morpheus", fontsize=36, y=1.05)
g.set_axis_labels("Layer", "Accuracy", fontsize=30)
g.set_titles(col_template="{col_name}", row_template="{row_name}", size=30)


Path(fig_path).mkdir(parents=True, exist_ok=True)
plt.savefig(f'{fig_path}/head_similarity_morpheus.png', dpi=300, bbox_inches='tight')
plt.savefig(f'{fig_path}/head_similarity_morpheus.pdf', dpi=300, bbox_inches='tight')

plt.show()